# TODO

# Import Libraries

In [19]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import sklearn
import pickle
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Code

## Import Dataset

In [5]:
path = 'Bone Break Classification'
class_list = os.listdir(path)
print(class_list)

['Avulsion fracture', 'Comminuted fracture', 'Fracture Dislocation', 'Greenstick fracture', 'Hairline Fracture', 'Impacted fracture', 'Longitudinal fracture', 'Oblique fracture', 'Pathological fracture', 'Spiral Fracture']


In [6]:
max_width = 0
max_height = 0

for (index, _class) in enumerate(class_list):
    class_path = os.path.join(path, _class)

    train_path = os.path.join(class_path, 'Train')
    for img_path in os.listdir(train_path):
        full_path = os.path.join(train_path, img_path)
        image = cv2.imread(full_path,0)
        height, width = image.shape[:2]
        max_width = max(max_width, width)
        max_height = max(max_height, height)

    test_path = os.path.join(class_path, 'Test')
    for img_path in os.listdir(test_path):
        full_path = os.path.join(test_path, img_path)
        image = cv2.imread(full_path,0)
        height, width = image.shape[:2]
        max_width = max(max_width, width)
        max_height = max(max_height, height)

print(f"Maximum width: {max_width}")
print(f"Maximum height: {max_height}")

max_size = max(max_height, max_width)
print(f"Max size: {max_size}x{max_size}")

Maximum width: 640
Maximum height: 640
Max size: 640x640


In [7]:
def resize_with_padding(image, target_size):
    old_size = image.shape[:2]  
    ratio = float(target_size) / max(old_size)  
    new_size = tuple([int(x * ratio) for x in old_size])  

    resized_image = cv2.resize(image, (new_size[1], new_size[0]))

    delta_w = target_size - new_size[1]
    delta_h = target_size - new_size[0]
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)
    
    color = [0, 0, 0]  
    padded_image = cv2.copyMakeBorder(resized_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    
    return padded_image

In [8]:
train_images, train_classes = [], []
test_images, test_classes = [], []

for (index, _class) in enumerate(class_list):
    class_path = os.path.join(path, _class)

    train_path = os.path.join(class_path, 'Train')
    for img_path in os.listdir(train_path):
        full_path = os.path.join(train_path, img_path)
        image = cv2.imread(full_path,0)
        resized_image = resize_with_padding(image, max_size)
        train_images.append(resized_image)
        train_classes.append(index)

    test_path = os.path.join(class_path, 'Test')
    for img_path in os.listdir(test_path):
        full_path = os.path.join(test_path, img_path)
        image = cv2.imread(full_path,0)
        resized_image = resize_with_padding(image, max_size)
        test_images.append(resized_image)
        test_classes.append(index)    

## Preprocessing

In [9]:
test_images = np.array(test_images)
train_images = np.array(train_images)
test_classes = np.array(test_classes)
train_classes = np.array(train_classes)

In [10]:
def normalize(image_list):
    return np.array([img.astype(np.float32) / 255.0 for img in image_list])

In [11]:
normalized_test_images = normalize(test_images)
normalized_train_images = normalize(train_images)
normalized_test_images.shape

(140, 640, 640)

In [12]:
total_class = len(set(train_classes))
train_classes_categorical = tf.keras.utils.to_categorical(train_classes, total_class)
test_classes_categorical = tf.keras.utils.to_categorical(test_classes, total_class)
# print(test_classes_categorical)
# test_classes_categorical.shape

In [13]:
X_train = normalized_train_images.reshape(len(normalized_train_images), -1)
X_test = normalized_test_images.reshape(len(normalized_test_images), -1)

In [14]:
y_train = np.argmax(train_classes_categorical, axis=-1)
y_test = np.argmax(test_classes_categorical, axis=-1)

## Model

### CNN

In [15]:
CNN_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=(3,3),
        activation='relu',
        input_shape=(max_size, max_size, 1)
    ),
    tf.keras.layers.MaxPool2D(
        pool_size=(2,2)
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(
        filters=64,
        kernel_size=(3,3),
        activation='relu'
    ),
    tf.keras.layers.MaxPool2D(
        pool_size=(2,2)
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        units=128, 
        activation='relu'
    ),
    tf.keras.layers.Dropout(
        rate=0.5
    ),
    tf.keras.layers.Dense(
        units=64, 
        activation='relu'
    ),
    tf.keras.layers.Dropout(
        rate=0.25
    ),
    tf.keras.layers.Dense(
        units=total_class, 
        activation='softmax'
    )
])

In [16]:
# CNN_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# CNN_history = CNN_model.fit(train_images, train_classes_categorical, epochs=10, batch_size = 32, validation_split=0.2)

In [17]:
# CNN_model.save('cnn_model.h5')

### ResNet50

In [18]:
resnet50_base = tf.keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(max_size, max_size, 1)
)
resnet50_base.trainable = False

resnet50_model = tf.keras.Sequential([
    resnet50_base,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        units=128, 
        activation='relu'
    ),
    tf.keras.layers.Dropout(
        rate=0.5
    ),
    tf.keras.layers.Dense(
        units=64, 
        activation='relu'
    ),
    tf.keras.layers.Dropout(
        rate=0.25
    ),
    tf.keras.layers.Dense(
        units=total_class, 
        activation='softmax'
    )
])

ValueError: The input must have 3 channels; Received `input_shape=(640, 640, 1)`

In [ ]:
# resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# resnet50_history = CNN_model.fit(train_images, train_classes_categorical, epochs=10, batch_size = 32, validation_split=0.2)

### SVC

In [ ]:
svc_model = SVC(
    kernel='linear',
    probability=True
)
svc_model.fit(X_train, y_train)
svc_prediction = svc_model.predict(X_test)
svc_report = classification_report(y_test, svc_prediction)
svc_accuracy = accuracy_score(y_test, svc_prediction)

In [ ]:
with open('svc_model.pkl', 'wb') as file:
    pickle.dump(svc_model, file)

### RF

In [ ]:
rf_model = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)
rf_model.fit(X_train, y_train)
rf_prediction = svc_model.predict(X_test)
rf_report = classification_report(y_test, rf_prediction)
rf_accuracy = accuracy_score(y_test, rf_prediction)

## Visualization